In [ ]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "law_meta": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
LEGAL_NOTES = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "law": safe_value(item.get(LAW_REFERENCES)),
            "legal_notes": safe_value(item.get(LEGAL_NOTES)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "law"}))
    
    return documents


# 문서화 실행
term_docs       = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs   = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
law_meta_docs   = convert_list_to_documents(load_json(file_paths["law_meta"]), "law_metadata")
car_case_docs   = convert_car_case_documents(load_json(file_paths["car_case"]))
law_docs        = convert_law_json_to_documents(load_json(file_paths["law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + law_meta_docs + law_docs

In [ ]:
import json
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

# car_case 문서 필터링 및 사고상황 추출
case_docs = [doc for doc in all_docs if doc.metadata.get("type") == "car_case"]
case_texts = [doc.metadata.get("situation", "") for doc in case_docs if doc.metadata.get("situation")]

# ko-sbert 임베딩
embed_model = SentenceTransformer("jhgan/ko-sbert-nli")
case_embeddings = embed_model.encode(case_texts)

# 사용자 입력
user_input = input("사고 상황을 입력하세요: ")
query_embedding = embed_model.encode([user_input])[0]

# 코사인 유사도 계산 및 Top-3 추출
cos_similarities = np.dot(case_embeddings, query_embedding) / (
    np.linalg.norm(case_embeddings, axis=1) * np.linalg.norm(query_embedding)
)
top_k_idx = np.argsort(cos_similarities)[-3:][::-1]
top_candidates = [case_docs[i] for i in top_k_idx]

# 판례 요약 출력
def summarize(doc, idx):
    return f"{idx+1}. 사건 ID: {doc.metadata.get('id')}\n사고상황: {doc.metadata.get('situation')}"

case_summaries = "\n\n".join([summarize(doc, i) for i, doc in enumerate(top_candidates)])

# GPT - 사건ID 선택(3개 중에 하나 판단)
selection_prompt = PromptTemplate(
    input_variables=["user_input", "case_summaries"],
    template="""
[사용자 입력 사고 상황]
{user_input}

[후보 판례 3건]
{case_summaries}

위 3건 중, 사고의 전개 구조(예: 직진 vs 좌회전, 도로 외 장소에서 진입, 교차로 내 진입 여부 등)가 사용자 상황과 가장 유사한 **사건 ID** 하나를 선택하세요.

반드시 다음 기준을 고려하세요:
- 차량들의 위치와 진입 경로가 유사한가?
- 사고 발생 지점과 방향이 유사한가?
- 각 차량의 신호·우선권 상황이 유사한가?|
- 도로 구조(교차로, 신호 유무, 도로 외 장소 등)가 유사한가?

출력 형식 (고정):
- 사건 ID: 차XX-X
- 판단 근거: (선택한 이유. 단순 유사성이 아니라, 어떤 지점이 유사했는지 명확히 설명할 것)
"""
)

llm = ChatOpenAI(model="gpt-4o", temperature=0)
selection_chain = LLMChain(llm=llm, prompt=selection_prompt)
selection_result = selection_chain.run(user_input=user_input, case_summaries=case_summaries)

# 사건 ID 파싱 및 선택
match = re.search(r"사건 ID[:：]?\s*(차\d{1,2}-\d{1,2})", selection_result)
selected_id = match.group(1) if match else None
selected_doc = next((doc for doc in case_docs if doc.metadata.get("id") == selected_id), None)

# 최종 판단 GPT 프롬프트(선택한 사건 object 내에서 과실비율 판단)
if selected_doc:
    # 해당 사건 관련 보조 문서들을 함께 전달
    related_docs = [doc for doc in all_docs if selected_id in doc.page_content]
    context_str = "\n\n".join(doc.page_content for doc in related_docs)

    final_prompt = PromptTemplate(
        input_variables=["user_input", "case_data"],
        template="""
너는 교통사고 과실 판단 전문가야.
아래 '사고 상황'을 분석하여 핵심 요소를 구조화하고, 반드시 문서 내에서 가장 유사한 사례(case)를 찾아 과실비율을 판단해줘.

---

사고 상황 원문:
{user_input}

➤ 사고 상황 요약 (다음 항목 기준):
- A차량 신호 및 진행 방식:
- B차량 신호 및 진행 방식:
- 충돌 방식 및 위치:
- 교차로/신호기 유무 등 도로 환경:

문서:
{case_data}

출력 형식 (고정):
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률:
   - [법률명] 제[조]조 [항]
4. 참고 판례:
   - [법원명] [사건번호]

조건:
- 반드시 문서 내 유사 사례를 기반으로 판단해야 해.
- 유사 사례와 현재 사고 상황이 정확히 일치하지 않으면, 차이점을 명시하고 과실비율 조정 이유를 설명해.
- 추측이나 상식은 사용하지 말고, 문서 정보만을 기반으로 판단해.
"""
    )

    final_chain = LLMChain(llm=llm, prompt=final_prompt)
    final_result = final_chain.run(user_input=user_input, case_data=context_str)

    print(f"\n선택된 사건 ID: {selected_id}")
    print("GPT 최종 판단 결과:\n")
    print(final_result)

else:
    print("\n❌ 사건 ID를 정확히 선택하지 못했습니다.")
    print("GPT 응답:\n", selection_result)

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings

# # 1. all_docs를 기반으로 벡터스토어 생성 (한 번만 수행하면 됨)
# embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")
# vectorstore = Chroma.from_documents(documents=all_docs, embedding=embedding_model)

# # 2. Retriever 생성
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# # 3. 프롬프트 정의
# detail_prompt = PromptTemplate(
#     template="""
# 아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

# 질문: {question}

# 문서: {context}

# 답변 형식:
# - 용어/조항 정의: [정확한 설명]
# - 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

# 답변:
# """,
#     input_variables=["question", "context"]
# )

# # 4. Retrieval QA 체인 구성
# detail_chain = RetrievalQA.from_chain_type(
#     llm=llm,  
#     retriever=retriever,
#     chain_type="stuff",
#     chain_type_kwargs={"prompt": detail_prompt}
# )

# # 5. 질문 실행
# question = "보행자우선도로가 뭔지 설명해줘"
# res2 = detail_chain.run(question)

# print("기능② 답변:\n", res2)

In [ ]:
# question = "판례 서울중앙지방법원 2020나59927에 대해 설명해줘"
# res2 = detail_chain.run(question)

# print("기능② 답변:\n", res2)